In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
pd.set_option('display.max_columns', None)

In [2]:
all_data = pd.read_csv("train/train.csv")
all_data['quarter_number'] = all_data['quarter'].str[-1]

In [3]:
rename_list = [('АДЫГЕЯ РЕСП', 'АДЫГЕЯ РЕСП.'),
 ('АЛТАЙСКИЙ КРАЙ.', 'АЛТАЙСКИЙ КРАЙ', 'АЛТАЙСКИЙ'),
 ('АЛЬШЕЕВСКИЙ Р-Н', 'АЛЬШЕЕВСКИЙ'),
 ('АРХАНГЕЛЬСКАЯ ОБЛ.', 'АРХАНГЕЛЬСКАЯ', 'АРХАНГЕЛЬСКИЙ', 'АРХАНГЕЛЬСКАЯ ОБЛ'),
 ('АУРГАЗИНСКИЙ', 'АУРГАЗИНСКИЙ Р-Н'),
 ('БАШКОРТОСТАН РЕСП.',
  'БАШКОРТОСТАН',
  'БАШКОРТОСТАН РЕСП',
  'РЕСП БАШКОРТОСТАН'),
 ('БЕЛАРУСЬ, МИНСКАЯ ОБЛ',
  'БЕЛАРУСЬ РЕСП',
  'БЕЛАРУСЬ РЕСП.',
  'БЕЛАРУСЬ'),
 ('БЕЛГОРОДСКАЯ', 'БЕЛГОРОДСКИЙ', 'БЕЛГОРОДСКАЯ ОБЛ'),
 ('БЕЛЕБЕЕВСКИЙ Р-Н', 'БЕЛЕБЕЕВСКИЙ'),
 ('БЕЛОРЕЦКИЙ', 'БЕЛОРЕЦКИЙ Р-Н'),
 ('БИЖБУЛЯКСКИЙ', 'БИЖБУЛЯКСКИЙ Р-Н'),
 ('БРЕСТСКАЯ ОБЛ', 'БРЕСТСКИЙ'),
 ('БРЯНСКАЯ', 'БРЯНСКАЯ ОБЛ', 'БРЯНСКАЯ ОБЛ.'),
 ('БРЯНСКАЯ ОБЛ', 'БРЯНСКАЯ ОБЛ.'),
 ('ВАСИЛЕОСТРОВСКИЙ', 'ЛОСИНООСТРОВСКИЙ'),
  ('ВОЛГОГРАДСКАЯ ОБЛ.', 'ВОЛГОГРАДСКАЯ ОБЛ', 'ВОЛГОГРАДСКАЯ'),
 ('ВОЛОГОДСКАЯ ОБЛ', 'ВОЛОГОДСКАЯ'),
 ('ВОРОНЕЖСКАЯ ОБЛ', 'ВОРОНЕЖСКАЯ'),
 ('Г МОСКВА', 'МОСКВА'),
 ('ГОМЕЛЬСКАЯ ОБЛ',
  'БЕЛАРУСЬ, ГОМЕЛЬСКАЯ ОБЛ',
  'ГОМЕЛЬСКАЯ ОБЛАСТЬ',
  'ГОМЕЛЬСКИЙ'),
 ('ДАВЛЕКАНОВСКИЙ Р-Н', 'ДАВЛЕКАНОВСКИЙ'),
 ('ДАГЕСТАН РЕСП', 'ДАГЕСТАН'),
 ('ДОНЕЦКАЯ НАРОДНАЯ РЕСП', 'ДОНЕЦКАЯ НАРОДНАЯ РЕСП.'),
 ('ДЮРТЮЛИНСКИЙ', 'ДЮРТЮЛИНСКИЙ Р-Н', 'ДЮРТЮЛИНСКИЙ РАЙОН'),
 ('ЕРМЕКЕЕВСКИЙ РАЙОН', 'ЕРМЕКЕЕВСКИЙ'),
 ('ЗАБАЙКАЛЬСКИЙ', 'ЗАБАЙКАЛЬСКИЙ КРАЙ.', 'ЗАБАЙКАЛЬСКИЙ КРАЙ'),
 ('ЗИАНЧУРИНСКИЙ', 'ЗИАНЧУРИНСКИЙ Р-Н'),
 ('ИВАНО-ФРАНКОВСКАЯ',
  'УКРАИНА, ИВАНО-ФРАНКОВСКАЯ ОБЛ.',
  'ИВАНО-ФРАНКОВСКАЯ ОБЛ'),
 ('ИГЛИНСКИЙ', 'ИГЛИНСКИЙ Р-Н'),
 ('ИЛИШЕВСКИЙ', 'ИЛИШЕВСКИЙ Р-Н'),
 ('ИНГУШЕТИЯ РЕСП', 'ИНГУШЕТИЯ'),
 ('ИРКУТСКАЯ ОБЛ', 'ИРКУТСКАЯ ОБЛ.'),
 ('ИРКУТСКАЯ ОБЛ', 'ИРКУТСКАЯ ОБЛ.', 'ИРКУТСКАЯ'),
 ('ИСИЛЬКУЛЬСКИЙ', 'ИСИКУЛЬСКИЙ'),
 ('ИШИМБАЙИСКИЙ', 'ИШИМБАЙСКИЙ Р-Н', 'ИШИМБАЙСКИЙ'),
 ('КАЗАХСТАН', 'КАЗАХСТАН, ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛ'),
 ('КАЛИНИНГРАДСКАЯ', 'КАЛИНИНГРАДСКАЯ ОБЛ'),
 ('КАЛТАСИНСКИЙ Р-Н', 'КАЛТАСИНСКИЙ'),
 ('КАМЧАТСКАЯ ОБЛ', 'КАМЧАТСКИЙ КРАЙ'),
 ('КАРАЧАЕВО-ЧЕРКЕССКАЯ', 'КАРАЧАЕВО-ЧЕРКЕССКАЯ РЕСП', 'КАРАЧАЕВСКИЙ'),
 ('КАРМАСКАЛИНСКИЙ Р-Н', 'КАРМАСКАЛИНСКИЙ'),
 ('КЕМЕРОВСКАЯ ОБЛАСТЬ - КУЗБАСС ОБЛ',
  'КЕМЕРОВСКАЯ ОБЛ',
  'КЕМЕРОВСКАЯ ОБЛАСТЬ - КУЗБАСС.',
  'КЕМЕРОВСКАЯ ОБЛАСТЬ - КУЗБАСС'),
 ('КИНЕЛЬ-ЧЕРКАССКИЙ', 'КИНЕЛЬ - ЧЕРКАССКИЙ'),
 ('КИРОВСКАЯ ОБЛ.', 'КИРОВСКАЯ ОБЛ', 'КИРОВСКАЯ', 'КИРОВСКИЙ'),
 ('КОМИ РЕСП.', 'КОМИ РЕСП'),
 ('КРАСНОДАРСКИЙ', 'КРАСНОДАРСКИЙ КРАЙ', 'КРАСНОДАРСКИЙ КРАЙ.'),
 ('КРАСНОКАМСКИЙ', 'КРАСНОКАМСКИЙ Р-Н'),
 ('КРАСНОСЕЛЬКУПСКИЙ', 'КРАСНОСЕЛЬСКИЙ'),
 ('КРАСНОЯРСКИЙ КРАЙ.', 'КРАСНОЯРСКИЙ', 'КРАСНОЯРСКИЙ КРАЙ'),
 ('КУРГАНСКАЯ ОБЛ.', 'КУРГАНСКАЯ ОБЛ', 'КУРГАНСКАЯ'),
 ('КУШНАРЕНКОВСКИЙ', 'КУШНАРЕНКОВСКИЙ Р-Н', 'КУШНАРЕВСКИЙ'),
 ('ЛЕНИНГРАДСКАЯ', 'ЛЕНИНГРАДСКАЯ ОБЛ.', 'ЛЕНИНГРАДСКАЯ ОБЛ'),
 ('ЛИПЕЦКАЯ ОБЛ', 'ЛИПЕЦКАЯ ОБЛ.', 'ЛИПЕЦКАЯ'),
 ('ЛУГАНСКАЯ', 'ЛУГАНСКАЯ НАРОДНАЯ РЕСП.'),
 ('МАРИЙ ЭЛ', 'МАРИЙ ЭЛ РЕСП', 'РЕСП МАРИЙ ЭЛ'),
 ('МЕЛЕУЗОВСКИЙ Р-Н', 'МЕЛЕУЗОВСКИЙ'),
 ('МОГИЛЁВСКАЯ ОБЛАСТЬ',
  'МОГИЛЕВСКАЯ ОБЛАСТЬ'),
 ('МИЯКИНСКИЙ', 'МИЯКИНСКИЙ Р-Н'),
 ('МОСКОВСКИЙ П', 'МОСКОВСКАЯ ОБЛ', 'МОСКОВСКАЯ', 'МОСКОВСКАЯ ОБЛ.'),
 ('НЕФТЕЮГАНСКИЙ Р-Н', 'НЕФТЕЮГАНСКИЙ'),
 ('НИЖЕГОРОДСКАЯ ОБЛ', 'НИЖЕГОРОДСКАЯ ОБЛ.'),
 ('НОВОСИБИРСКАЯ ОБЛ', 'НОВОСИБИРСКАЯ', 'НОВОСИБИРСКАЯ ОБЛ.'),
 ('ОРЕНБУРГСКАЯ', 'ОРЕНБУРГСКАЯ ОБЛ.', 'ОРЕНБУРГСКАЯ ОБЛ'),
 ('ОРЛОВСКАЯ ОБЛ', 'ОРЛОВСКАЯ ОБЛ.'),
 ('ПЕНЗЕНСКАЯ ОБЛ', 'ПЕНЗЕНСКАЯ', 'ПЕНЗЕНСКАЯ ОБЛ.'),
 ('ПЕРМСКИЙ', 'ПЕРМСКИЙ КРАЙ.', 'ПЕРМСКИЙ КРАЙ'),
 ('ПОСЕЛЕНИЕ ВНУКОВСКОЕ', 'П.ВНУКОВСКОЕ', 'ВНУКОВСКОЕ П'),
 ('ПРИМОРСКИЙ Р-Н', 'ПРИМОРСКИЙ', 'ПРИМОРСКИЙ КРАЙ.', 'ПРИМОРСКИЙ КРАЙ'),
 ('РОСТОВСКАЯ ОБЛ', 'РОСТОВСКАЯ'),
 ('РЯЗАНСКАЯ ОБЛ', 'РЯЗАНСКАЯ ОБЛ.', 'РЯЗАНСКАЯ'),
 ('САМАРСКАЯ', 'САМАРСКАЯ ОБЛ', 'САМАРСКАЯ ОБЛ.'),
 ('САНКТ-ПЕТЕРБУРГ Г', 'САНКТ-ПЕТЕРБУРГ', 'Г САНКТ-ПЕТЕРБУРГ'),
 ('САХАЛИНСКАЯ ОБЛ', 'САХАЛИНСКАЯ'),
 ('СВЕРДЛОВСКАЯ ОБЛ.', 'СВЕРДЛОВСКАЯ ОБЛ', 'СВЕРДЛОВСКАЯ'),
 ('СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ РЕСП',
  'СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ',
  'РЕСП СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ',
  'СЕВЕРНАЯ ОСЕТИЯ-АЛАНИЯ РЕСП'),
 ('СОСЕНСКОЕ П', 'СОСЕНСКОЕ'),
 ('СТАВРОПОЛЬСКИЙ', 'СТАВРОПОЛЬСКИЙ КРАЙ', 'СТАВРОПОЛЬСКИЙ КРАЙ.'),
 ('СТЕРЛИБАШЕВСКИЙ', 'СТЕРЛИБАШЕВСКИЙ Р-Н'),
 ('ТАМБОВСКАЯ', 'ТАМБОВСКАЯ ОБЛ'),
 ('ТАТАРСТАН РЕСП', 'ТАТАРСТАН', 'РЕСП ТАТАРСТАН', 'ТАТАРСТАН РЕСП.'),
 ('ТАТЫШЛИНСКИЙ Р-Н', 'ТАТЫШЛИНСКИЙ'),
 ('ТОМСКАЯ', 'ТОМСКАЯ ОБЛ', 'ТОМСКАЯ ОБЛ.'),
 ('ТУЙМАЗИНСКИЙ Р-Н', 'ТУЙМАЗИНСКИЙ'),
 ('ТЮМЕНСКАЯ', 'ТЮМЕНСКАЯ ОБЛ', 'ТЮМЕНСКАЯ ОБЛ.'),
 ('УДМУРТСКАЯ РЕСП.', 'УДМУРТСКАЯ РЕСП', 'УДМУРТСКАЯ'),
 ('УКРАИНА', 'УКРАИНА, ИВАНО-ФРАНКОВСКАЯ ОБЛ.'),
 ('УЛЬЯНОВСКАЯ', 'УЛЬЯНОВСКАЯ ОБЛ', 'УЛЬЯНОВСКАЯ ОБЛ.'),
 ('ХАБАРОВСКИЙ КРАЙ', 'ХАБАРОВСКИЙ КРАЙ.', 'ХАБАРОВСКИЙ'),
 ('ХАКАСИЯ', 'ХАКАСИЯ РЕСП', 'РЕСП ХАКАСИЯ', 'ХАКАСИЯ РЕСП.'),
 ('ХАКАСИЯ РЕСП', 'ХАКАСИЯ РЕСП.'),
 ('ХАНТЫ-МАНСИЙСКИЙ',
  'ХАНТЫ-МАНСИЙСКИЙ-ЮГРА АО',
  'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА',
  'ХАНТЫ-МАНСИЙСКИЙ АО',
  'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ-ЮГРА АО',
  'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ',
  'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ АО',
  'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА АО.',
  'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА АО'),
 ('ЧЕЛЯБИНСКАЯ ОБЛ.', 'ЧЕЛЯБИНСКАЯ ОБЛ', 'ЧЕЛЯБИНСКАЯ'),
 ('ЧЕЧЕНСКАЯ РЕСП.', 'ЧЕЧЕНСКАЯ', 'ЧЕЧЕНСКАЯ РЕСП'),
 ('ЧИШМИНСКИЙ', 'ЧИШМИНСКИЙ Р-Н'),
 ('ЧУВАШСКАЯ РЕСП',
  'ЧУВАШСКАЯ РЕСПУБЛИКА -',
  'ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ',
  'ЧУВАШСКАЯ РЕСПУБЛИКА ЧУВАШИЯ'),
 ('ЯМАЛО-НЕНЕЦКИЙ АВТОНОМНЫЙ ОКРУГ',
  'НЕНЕЦКИЙ АО',
  'ЯМАЛО-НЕНЕЦКИЙ А.ОКР.',
  'ЯМАЛО-НЕНЕЦКИЙ',
  'ЯМАЛО-НЕНЕЦКИЙ АО.',
  'ЯМАЛО-НЕНЕЦКИЙ АО'),
 ('ЯНАУЛЬСКИЙ', 'ЯНАУЛЬСКИЙ Р-Н')]

In [11]:
npo_checks_data = pd.read_csv("train/cntrbtrs.csv", sep=";")[['npo_accnt_id', 'accnt_pnsn_schm']].rename(columns={'npo_accnt_id': 'npo_account_id'}) # содержит информацию о счетах НПО
all_data = pd.merge(all_data, npo_checks_data, how="left", on=["npo_account_id"])

all_data['accnt_pnsn_schm'] = all_data['accnt_pnsn_schm'].astype(int)

KeyError: 'accnt_pnsn_schm'

In [ ]:
rename_dict = {}
for names in rename_list:
    for name in names[1:]:
        rename_dict[name] = names[0]

In [ ]:
rename_dict['МОСКВА Г.'] = 'МОСКВА'
rename_dict['МОСКВА Г'] = 'МОСКВА'
rename_dict['Г МОСКВА'] = 'МОСКВА'
rename_dict['ОКРУГ Г. МОСКВЫ'] = 'МОСКВА'


In [ ]:
print(all_data['region'].nunique())
all_data['region'] = all_data['region'].replace(rename_dict)
print(all_data['region'].nunique())

448
286


In [12]:
all_data = all_data[~(all_data['region'].isin(['УКРАИНА', '2 МКР', 'ИВАНО-ФРАНКОВСКАЯ', 
                    'СОЕДИНЕННОЕ КОРОЛЕВСТВО', 'ГЕРМАНИЯ', '-', '77', 'ОБЪЕДИНЕННЫЕ АРАБСКИЕ ЭМИРАТЫ', ]))]

all_data["quarter_number"] = all_data["quarter_number"].astype(int)

/var/folders/pd/dc92r6ps4_q25drxrf_7btlm0000gn/T/ipykernel_37397/2688506868.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data["quarter_number"] = all_data["quarter_number"].astype(int)


In [14]:
feats = ['npo_accnts_nmbr', 'pmnts_type', 'gender', 'age', 'clnt_cprtn_time_d',
       'actv_prd_d', 'lst_pmnt_rcnc_d', 'balance', 'oprtn_sum_per_qrtr',
       'frst_pmnt', 'lst_pmnt', 'pmnts_sum', 'pmnts_nmbr',
       'pmnts_sum_per_qrtr', 'pmnts_nmbr_per_qrtr', 'incm_sum', 'incm_per_qrtr',
       'mgd_accum_period', 'mgd_payment_period', 'phone_number', 'email', 'lk',
       'assignee_npo', 'assignee_ops', 'citizen',
       'fact_addrss', 'appl_mrkr', 'evry_qrtr_pmnt', 'oprtn_sum_per_year', 'pmnts_sum_per_year', 'pmnts_nmbr_per_year', 'incm_per_year']

In [15]:
from datetime import datetime

def adding_lags(train_df, lag, feats, sort_tables=['year', 'quarter_number'], plus_one=False):
    feats_lag = []
    for feat in feats:
        feats_lag.append(feat + f'_lag_{lag}_quartal')

    if plus_one: lag+=1
    train_df[feats_lag] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])[feats].shift(lag)
    return train_df

def adding_lags_of_churn(train_df, sort_tables=['year', 'quarter_number']):
    train_df["churn"] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])["churn"].shift(-1)
    train_df = train_df[train_df["churn"].notna()]
    return train_df

def adding_lags_of_feats_for_churn(train_df, feats, sort_tables=['year', 'quarter_number']):
    train_df[feats] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])[feats].shift(1)
    return train_df

def get_data_period(agg_df, start_year, target_year):
    start_date_mask_agg = agg_df['year'] >= start_year
    end_date_mask_agg = agg_df['year'] < target_year
    filtered_agg_df = agg_df[start_date_mask_agg & end_date_mask_agg]

    targets = agg_df[agg_df['year'] == target_year]

    temp_mask_q1 = targets['quarter'].str
    target_list = [targets[targets['quarter'].str.contains(f'Q{i}')] for i in range(1, 5)]
    return filtered_agg_df, target_list

In [20]:
# train_val_df, test_df = get_data_period(all_data, start_year=2008, target_year=2022)
# train_df, val_df = get_data_period(train_val_df, start_year=2008, target_year=2021)
# train_df.shape
train_df = all_data
# train_df, test_df = get_data_period(all_data, start_year=2008, target_year=2022)

In [23]:
# для выделения случаев когда прошлый период был с 1, а затем стал 0
train_df['last_churn'] = train_df.sort_values(['year', 'quarter_number'], ascending=[True, True]).groupby(['npo_account_id'])['churn'].shift(1)
unchurned_after_churn = train_df[train_df['last_churn'] == 1]

In [22]:
train_df = train_df.drop(index=unchurned_after_churn.index, columns='last_churn')
train_df.shape

(3589026, 45)

In [52]:
# train_df = adding_lags_of_churn(train_df)
# train_df.shape

(2285755, 44)

In [24]:
lags = [1, 2, 4, 8, 12]
for lag_number in lags:
    train_df = adding_lags(train_df, lag_number, feats)

/var/folders/pd/dc92r6ps4_q25drxrf_7btlm0000gn/T/ipykernel_37397/1484430889.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[feats_lag] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])[feats].shift(lag)
/var/folders/pd/dc92r6ps4_q25drxrf_7btlm0000gn/T/ipykernel_37397/1484430889.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[feats_lag] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])[feats].shift(lag)
/var/folders/pd/dc92r6ps4_

In [27]:
# Добавить обработку frst_pmnt_date и lst_pmnt_date_per_qrtr

categorical_features = ["region", "accnt_pnsn_schm"]
# categorical_features = ["region", "npo_accnts_nmbr", 'pmnts_type', 'gender', 'phone_number', 'email', 'lk', 'assignee_npo', 'assignee_ops', 'citizen', 'fact_addrss', 'appl_mrkr', 'evry_qrtr_pmnt']
# ignored_features = ["slctn_nmbr", "client_id", "npo_account_id", "year", "quarter",
#                      "frst_pmnt_date", "lst_pmnt_date_per_qrtr", 'oprtn_sum_per_year', 'pmnts_sum_per_year', 'pmnts_nmbr_per_year', 'incm_per_year']
ignored_features = ["slctn_nmbr", "client_id", "npo_account_id", "year", "quarter",
                     "frst_pmnt_date", "lst_pmnt_date_per_qrtr"]

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df[train_df["year"] >= 2004].drop(columns="churn").drop(columns=ignored_features), train_df[train_df["year"] >= 2004]["churn"], test_size=0.15)

In [28]:
# X_train, y_train = train_df[train_df["year"] >= 2004].drop(columns="churn").drop(columns=ignored_features), train_df[train_df["year"] >= 2004]["churn"]

In [18]:
# feats_to_swap = list(set(test_df[0].columns) - set(['churn', 'quarter_number', 'year']))

# Xy_val = pd.concat([val_df[0], val_df[1], val_df[2], val_df[3]])

# val_quart_index = Xy_val.index
# data_for_Lag = pd.concat([Xy_val, train_df])
# data_for_Lag = adding_lags_of_feats_for_churn(data_for_Lag, feats_to_swap)
# for lag_number in lags:
#     data_for_Lag = adding_lags(data_for_Lag, lag_number, feats, plus_one=True)
    
# val_df = data_for_Lag.loc[val_quart_index]

In [19]:
# Xy_val = pd.concat([val_df[0], val_df[1], val_df[2], val_df[3]])
# X_val, y_val = val_df.drop(columns="churn").drop(columns=ignored_features), val_df["churn"]

In [31]:
# X_val['accnt_pnsn_schm'] = X_val['accnt_pnsn_schm'].fillna(-1).astype(int)
# X_train['accnt_pnsn_schm'] = X_train['accnt_pnsn_schm'].fillna(-1).astype(int)

In [33]:
model = CatBoostClassifier(random_seed=42, iterations=500, learning_rate=0.01)
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(n_estimators = 50)

In [38]:
X_train['region'] = X_train['region'].fillna('None')
X_val['region'] = X_val['region'].fillna('None')

In [23]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)  
X_resampled.shape, X_train.shape

((65790, 172), (1996583, 172))

In [25]:
# X_train.reset_index().to_feather('X_train.feather')
# y_train.reset_index().to_feather('y_train.feather')

# X_val.reset_index().to_feather('X_val.feather')
# y_val.reset_index().to_feather('y_val.feather')

In [26]:
# model.fit(X_train.drop(columns='region').fillna(-10), y_train)

In [40]:
model.fit(
    X_train, y_train,
    cat_features=['region'],
    eval_set=(X_val, y_val), 
    early_stopping_rounds=10
);

0:	learn: 0.6636398	test: 0.6636566	best: 0.6636566 (0)	total: 1.4s	remaining: 11m 38s
1:	learn: 0.6366642	test: 0.6366923	best: 0.6366923 (1)	total: 2.31s	remaining: 9m 34s
2:	learn: 0.6113437	test: 0.6113704	best: 0.6113704 (2)	total: 3.21s	remaining: 8m 51s
3:	learn: 0.5894221	test: 0.5894433	best: 0.5894433 (3)	total: 4.35s	remaining: 8m 59s
4:	learn: 0.5662461	test: 0.5662754	best: 0.5662754 (4)	total: 5.28s	remaining: 8m 42s
5:	learn: 0.5465720	test: 0.5465986	best: 0.5465986 (5)	total: 6.11s	remaining: 8m 23s
6:	learn: 0.5244960	test: 0.5245336	best: 0.5245336 (6)	total: 7.13s	remaining: 8m 22s
7:	learn: 0.5060020	test: 0.5060273	best: 0.5060273 (7)	total: 8.09s	remaining: 8m 17s
8:	learn: 0.4864229	test: 0.4864453	best: 0.4864453 (8)	total: 9.05s	remaining: 8m 13s
9:	learn: 0.4671929	test: 0.4672187	best: 0.4672187 (9)	total: 9.88s	remaining: 8m 4s
10:	learn: 0.4487913	test: 0.4488233	best: 0.4488233 (10)	total: 10.7s	remaining: 7m 54s
11:	learn: 0.4316765	test: 0.4317122	best:

In [41]:
roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

0.9698267559703916

In [29]:
preds = model.predict_proba(X_val)[:, 1]
y_pred_binary = (preds > 0.09).astype(int)
f1 = f1_score(y_val, y_pred_binary)
f1

0.028442864285317483

In [30]:
feats_to_swap = list(set(test_df[0].columns) - set(['churn', 'quarter_number', 'year']))
Xy_test = pd.concat([test_df[0], test_df[1], test_df[2], test_df[3]])

test_quart_index = Xy_test.index
data_for_Lag = pd.concat([Xy_test, val_df, train_df])
data_for_Lag = adding_lags_of_feats_for_churn(data_for_Lag, feats_to_swap)

for lag_number in lags:
    ata_for_Lag = adding_lags(data_for_Lag, lag_number, feats, plus_one=True)
    
test_df  = data_for_Lag.loc[test_quart_index]

In [31]:
val_df.shape

(262836, 184)

In [32]:
val_df['churn'].sum()

4084.0

In [33]:
X_test, y_test = test_df.drop(columns="churn").drop(columns=ignored_features), test_df["churn"]
X_test['accnt_pnsn_schm'] = X_test['accnt_pnsn_schm'].fillna(-1).astype(int)
X_test['region'] = X_test['region'].fillna('None')

preds = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, preds[:, 1])
y_pred_binary = (preds[:, 1] > 0.09).astype(int)
f1 = f1_score(y_test, y_pred_binary)
    
print(roc_auc, f1)

0.6051946516032412 0.01905795790362463


In [34]:
roc_auc_score(y_val, model.predict(X_val))

0.502800407979548

In [35]:
importances = model.get_feature_importance(type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
feature_importances

oprtn_sum_per_qrtr                14.614092
pmnts_type                         7.558581
pmnts_type_lag_1_quartal           5.890126
pmnts_type_lag_2_quartal           4.025056
mgd_payment_period                 2.808055
                                    ...    
balance_lag_8_quartal              0.000000
assignee_ops                       0.000000
actv_prd_d_lag_12_quartal          0.000000
lst_pmnt_rcnc_d_lag_12_quartal     0.000000
email_lag_4_quartal                0.000000
Length: 172, dtype: float64

In [49]:
score_df = pd.read_csv('test/test.csv')
score_df

,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,lst_pmnt_rcnc_d,balance,oprtn_sum_per_qrtr,oprtn_sum_per_year,frst_pmnt_date,lst_pmnt_date_per_qrtr,frst_pmnt,lst_pmnt,pmnts_sum,pmnts_nmbr,pmnts_sum_per_qrtr,pmnts_sum_per_year,pmnts_nmbr_per_qrtr,pmnts_nmbr_per_year,incm_sum,incm_per_qrtr,incm_per_year,mgd_accum_period,mgd_payment_period,phone_number,email,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt
0,0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,4543,31,464770.58,10880.44,39763.52,2004-12-31,2017-06-09,19164.90,4168.46,342134.88,151,10880.43,39763.52,3,11,122635.69,-0.00,-0.00,0.0,0.0,-1,0,1,-1,-1,398046.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,1
1,2,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,0,2209,221.24,10.54,10.54,2015-12-14,NaN,82.48,82.48,82.48,1,-0.00,-0.00,0,0,138.75,10.53,10.53,5.0,5.0,-1,-1,-1,-1,-1,0.0,МОСКВА Г,1,1,0,0
2,3,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,1615,29,27071.14,1850.63,8540.05,2011-04-13,2015-09-14,301.96,948.37,24972.87,54,1850.63,6401.21,3,10,2098.26,-0.00,2138.83,0.0,0.0,0,0,-1,-1,-1,666211.0,ИРКУТСКАЯ ОБЛ,-1,1,0,0
3,2,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,3538,30,50580.42,1219.51,6860.62,2005-12-31,2015-09-08,8455.28,406.50,31951.22,118,1219.51,4878.04,3,12,13909.69,-0.00,1982.57,4.0,4.0,-1,-1,-1,-1,-1,162610.0,ВОЛОГОДСКАЯ ОБЛ,1,1,0,1
4,0,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,276,30,7300.73,1747.83,9468.41,2010-12-09,2011-09-11,301.83,1353.97,7299.96,10,1747.83,9220.26,3,12,0.76,-0.00,248.13,0.0,0.0,-1,0,-1,-1,-1,427111.0,УДМУРТСКАЯ РЕСП,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633429,0,0x166618F0B41D4744B2B1AF9983F41ED7,0x3086DA482BB98F4CB01FBFBCF699B603,1,3,2022,2022Q3,-1,55,2510,2430,28,301321.98,3298.51,20151.59,2015-11-16,2022-07-12,2659.61,3298.51,254564.78,81,3298.51,20151.59,1,7,46757.19,-0.00,-0.00,0.0,0.0,-1,0,1,-1,-1,352800.0,КРАСНОДАРСКИЙ КРАЙ,1,1,0,1
633430,0,0xB9C2A95250AE0846A0AAA02B8EC5D717,0x44CA6910C3C5904BA87936C53926BABC,1,3,2020,2020Q2,1,52,1580,1491,29,383896.62,23074.76,97507.81,2016-05-11,2020-06-10,8893.30,250.66,343584.73,52,23074.75,82051.25,3,12,40311.87,-0.00,15456.55,0.0,0.0,-1,0,-1,-1,-1,614016.0,ПЕРМСКИЙ КРАЙ,1,1,0,1
633431,0,0xA629005056964A2C11ED63E9F4E5512F,0xA62A005056964A2C11ED7C7CECEF8EC0,1,3,2022,2022Q4,1,49,382,365,29,38564.94,8398.63,36030.92,2021-12-14,2022-12-14,2527.57,2676.06,38558.49,14,8398.62,36030.92,3,13,6.44,-0.00,-0.00,0.0,0.0,-1,0,-1,-1,-1,628331.0,ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА АО,-1,-1,0,1
633432,0,0x4D4971EE22A08249BE6F688738DB4D97,0xABCE12A6F911374E96C7CB080F45C900,1,3,2021,2021Q2,1,33,890,825,21,19998.37,1764.69,10635.34,2019-03-13,2021-06-15,402.99,725.38,19403.32,28,1764.69,9493.61,3,13,595.04,-0.00,1141.73,0.0,0.0,-1,0,-1,-1,-1,665824.0,ИРКУТСКАЯ ОБЛ,1,1,0,1


In [50]:
# без этого ругается
score_df.index = score_df.index + 4_000_000

In [51]:
all_data
test_quart_index = score_df.index
data_for_Lag = pd.concat([all_data.drop(columns="churn"), score_df]) 

for lag_number in lags:
    data_for_Lag = adding_lags(data_for_Lag, lag_number, feats)
    
score_df = data_for_Lag.loc[test_quart_index]

/var/folders/pd/dc92r6ps4_q25drxrf_7btlm0000gn/T/ipykernel_37397/1484430889.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[feats_lag] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])[feats].shift(lag)
/var/folders/pd/dc92r6ps4_q25drxrf_7btlm0000gn/T/ipykernel_37397/1484430889.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[feats_lag] = train_df.sort_values(sort_tables, ascending=[True, True]).groupby(['npo_account_id'])[feats].shift(lag)
/var/folders/pd/dc92r6ps4_

In [52]:
score_df

,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,lst_pmnt_rcnc_d,balance,oprtn_sum_per_qrtr,oprtn_sum_per_year,frst_pmnt_date,lst_pmnt_date_per_qrtr,frst_pmnt,lst_pmnt,pmnts_sum,pmnts_nmbr,pmnts_sum_per_qrtr,pmnts_sum_per_year,pmnts_nmbr_per_qrtr,pmnts_nmbr_per_year,incm_sum,incm_per_qrtr,incm_per_year,mgd_accum_period,mgd_payment_period,phone_number,email,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,quarter_number,accnt_pnsn_schm_x,accnt_pnsn_schm_y,last_churn,npo_accnts_nmbr_lag_1_quartal,pmnts_type_lag_1_quartal,gender_lag_1_quartal,age_lag_1_quartal,clnt_cprtn_time_d_lag_1_quartal,actv_prd_d_lag_1_quartal,lst_pmnt_rcnc_d_lag_1_quartal,balance_lag_1_quartal,oprtn_sum_per_qrtr_lag_1_quartal,frst_pmnt_lag_1_quartal,lst_pmnt_lag_1_quartal,pmnts_sum_lag_1_quartal,pmnts_nmbr_lag_1_quartal,pmnts_sum_per_qrtr_lag_1_quartal,pmnts_nmbr_per_qrtr_lag_1_quartal,incm_sum_lag_1_quartal,incm_per_qrtr_lag_1_quartal,mgd_accum_period_lag_1_quartal,mgd_payment_period_lag_1_quartal,phone_number_lag_1_quartal,email_lag_1_quartal,lk_lag_1_quartal,assignee_npo_lag_1_quartal,assignee_ops_lag_1_quartal,citizen_lag_1_quartal,fact_addrss_lag_1_quartal,appl_mrkr_lag_1_quartal,evry_qrtr_pmnt_lag_1_quartal,oprtn_sum_per_year_lag_1_quartal,pmnts_sum_per_year_lag_1_quartal,pmnts_nmbr_per_year_lag_1_quartal,incm_per_year_lag_1_quartal,npo_accnts_nmbr_lag_2_quartal,pmnts_type_lag_2_quartal,gender_lag_2_quartal,age_lag_2_quartal,clnt_cprtn_time_d_lag_2_quartal,actv_prd_d_lag_2_quartal,lst_pmnt_rcnc_d_lag_2_quartal,balance_lag_2_quartal,oprtn_sum_per_qrtr_lag_2_quartal,frst_pmnt_lag_2_quartal,lst_pmnt_lag_2_quartal,pmnts_sum_lag_2_quartal,pmnts_nmbr_lag_2_quartal,pmnts_sum_per_qrtr_lag_2_quartal,pmnts_nmbr_per_qrtr_lag_2_quartal,incm_sum_lag_2_quartal,incm_per_qrtr_lag_2_quartal,mgd_accum_period_lag_2_quartal,mgd_payment_period_lag_2_quartal,phone_number_lag_2_quartal,email_lag_2_quartal,lk_lag_2_quartal,assignee_npo_lag_2_quartal,assignee_ops_lag_2_quartal,citizen_lag_2_quartal,fact_addrss_lag_2_quartal,appl_mrkr_lag_2_quartal,evry_qrtr_pmnt_lag_2_quartal,oprtn_sum_per_year_lag_2_quartal,pmnts_sum_per_year_lag_2_quartal,pmnts_nmbr_per_year_lag_2_quartal,incm_per_year_lag_2_quartal,npo_accnts_nmbr_lag_4_quartal,pmnts_type_lag_4_quartal,gender_lag_4_quartal,age_lag_4_quartal,clnt_cprtn_time_d_lag_4_quartal,actv_prd_d_lag_4_quartal,lst_pmnt_rcnc_d_lag_4_quartal,balance_lag_4_quartal,oprtn_sum_per_qrtr_lag_4_quartal,frst_pmnt_lag_4_quartal,lst_pmnt_lag_4_quartal,pmnts_sum_lag_4_quartal,pmnts_nmbr_lag_4_quartal,pmnts_sum_per_qrtr_lag_4_quartal,pmnts_nmbr_per_qrtr_lag_4_quartal,incm_sum_lag_4_quartal,incm_per_qrtr_lag_4_quartal,mgd_accum_period_lag_4_quartal,mgd_payment_period_lag_4_quartal,phone_number_lag_4_quartal,email_lag_4_quartal,lk_lag_4_quartal,assignee_npo_lag_4_quartal,assignee_ops_lag_4_quartal,citizen_lag_4_quartal,fact_addrss_lag_4_quartal,appl_mrkr_lag_4_quartal,evry_qrtr_pmnt_lag_4_quartal,oprtn_sum_per_year_lag_4_quartal,pmnts_sum_per_year_lag_4_quartal,pmnts_nmbr_per_year_lag_4_quartal,incm_per_year_lag_4_quartal,npo_accnts_nmbr_lag_8_quartal,pmnts_type_lag_8_quartal,gender_lag_8_quartal,age_lag_8_quartal,clnt_cprtn_time_d_lag_8_quartal,actv_prd_d_lag_8_quartal,lst_pmnt_rcnc_d_lag_8_quartal,balance_lag_8_quartal,oprtn_sum_per_qrtr_lag_8_quartal,frst_pmnt_lag_8_quartal,lst_pmnt_lag_8_quartal,pmnts_sum_lag_8_quartal,pmnts_nmbr_lag_8_quartal,pmnts_sum_per_qrtr_lag_8_quartal,pmnts_nmbr_per_qrtr_lag_8_quartal,incm_sum_lag_8_quartal,incm_per_qrtr_lag_8_quartal,mgd_accum_period_lag_8_quartal,mgd_payment_period_lag_8_quartal,phone_number_lag_8_quartal,email_lag_8_quartal,lk_lag_8_quartal,assignee_npo_lag_8_quartal,assignee_ops_lag_8_quartal,citizen_lag_8_quartal,fact_addrss_lag_8_quartal,appl_mrkr_lag_8_quartal,evry_qrtr_pmnt_lag_8_quartal,oprtn_sum_per_year_lag_8_quartal,pmnts_sum_per_year_lag_8_quartal,pmnts_nmbr_per_year_lag_8_quartal,in

In [10]:
score_df = pd.read_csv('test/sample_submission.csv')
score_df

,npo_account_id,quarter,churn
0,0x0000132B2D126446B3E105530BA834B9,2009Q2,0
1,0x0000132B2D126446B3E105530BA834B9,2012Q1,0
2,0x0000132B2D126446B3E105530BA834B9,2017Q1,0
3,0x0000132B2D126446B3E105530BA834B9,2017Q2,0
4,0x0000132B2D126446B3E105530BA834B9,2020Q2,0
...,...,...,...
633429,0xFFFF57AF7B56DE4891B4CD4E78A5B892,2020Q4,0
633430,0xFFFF5DA61B60574194D4B977023B15B9,2009Q4,0
633431,0xFFFF5DA61B60574194D4B977023B15B9,2017Q4,0
633432,0xFFFFF59735AAF14E9DABA46C84CAD40B,2017Q3,0


In [36]:
import shap

shap.initjs()

In [42]:
sample_number = 300_000
explainer = shap.TreeExplainer(model)
shap_values = explainer(X_train.tail(sample_number))

shap.plots.beeswarm(shap_values, max_display=30)

In [ ]:
preds = model.predict_proba(X_train.tail(sample_number))[:, 1]

In [ ]:
np.argpartition(preds, -1000)[-1000:-900], preds[269363]

In [ ]:
pd.DataFrame(train_df.iloc[269363])

In [ ]:
shap.plots.force(shap_values[269363, ...])

In [ ]:
train_df.groupby('pmnts_nmbr_per_qrtr')['churn'].sum()

In [ ]:
train_df.groupby('pmnts_type')['churn'].mean()

In [ ]:
train_df.iloc[115666]['churn']

In [ ]:
shap.plots.beeswarm(shap_values[preds > 0.5], max_display=10)

In [ ]:
shap.plots.beeswarm(shap_values[preds < 0.09], max_display=10)

In [ ]:
shap_y = y_train.tail(100_00).reset_index()
shap_y[shap_y['churn'] == 1].iloc[:5]